In [1]:
import torch
import torch.nn.functional as F
import matplotlib.pyplot as plt
%matplotlib inline

In [3]:
words = open('names.txt', 'r').read().splitlines()
words[:8]

['emma', 'olivia', 'ava', 'isabella', 'sophia', 'charlotte', 'mia', 'amelia']

In [5]:
#build the vocabulary of characters and the mappings to/from characters
chars = sorted(list(set(''.join(words))))
stoi = {s:i+1 for i, s in enumerate(chars)}
stoi['.'] = 0
itos = {i:s for s, i in stoi.items()}
print(itos)

{1: 'a', 2: 'b', 3: 'c', 4: 'd', 5: 'e', 6: 'f', 7: 'g', 8: 'h', 9: 'i', 10: 'j', 11: 'k', 12: 'l', 13: 'm', 14: 'n', 15: 'o', 16: 'p', 17: 'q', 18: 'r', 19: 's', 20: 't', 21: 'u', 22: 'v', 23: 'w', 24: 'x', 25: 'y', 26: 'z', 0: '.'}


In [130]:
#build the dataset
block_size = 3
X, Y = [], []
for w in words:
    #print(w)
    context = [0]*block_size
    for ch in w + '.':
        ix = stoi[ch]
        X.append(context)
        Y.append(ix)
        #print(''.join(itos[i] for i in context), '---->', itos[ix])
        context = context[1:] + [ix]

X = torch.tensor(X)
Y = torch.tensor(Y)

In [7]:
# build the dataset
block_size = 3 # context length: how many characters do we take to predict the next one?

def build_dataset(words):  
  X, Y = [], []
  for w in words:

    #print(w)
    context = [0] * block_size
    for ch in w + '.':
      ix = stoi[ch]
      X.append(context)
      Y.append(ix)
      #print(''.join(itos[i] for i in context), '--->', itos[ix])
      context = context[1:] + [ix] # crop and append

  X = torch.tensor(X)
  Y = torch.tensor(Y)
  print(X.shape, Y.shape)
  return X, Y

import random
random.seed(42)
random.shuffle(words)
n1 = int(0.8*len(words))
n2 = int(0.9*len(words))

Xtr, Ytr = build_dataset(words[:n1])
Xdev, Ydev = build_dataset(words[n1:n2])
Xte, Yte = build_dataset(words[n2:])

torch.Size([182625, 3]) torch.Size([182625])
torch.Size([22655, 3]) torch.Size([22655])
torch.Size([22866, 3]) torch.Size([22866])


In [131]:
X.shape, X.dtype, Y.shape, Y.dtype
#X is the input like - three characters and theres 32 combinations in the dataset as printed above and the 2nd dimension i.e. 3 is the indices 
#of those characters
#Y is the output - single index pointing to the next character in the sequence

(torch.Size([228146, 3]), torch.int64, torch.Size([228146]), torch.int64)

In [6]:
C = torch.randn([27, 2])
C

tensor([[-0.4911, -0.7366],
        [-1.0913,  1.6298],
        [-1.2784,  1.0615],
        [ 0.7025, -0.3080],
        [-1.1638, -0.5904],
        [-0.3637, -0.6524],
        [-0.7265, -1.0318],
        [ 0.0297,  0.9750],
        [-0.6060, -0.0844],
        [ 0.4534, -1.4865],
        [ 0.2807,  0.3752],
        [ 0.1690, -1.6043],
        [ 1.5809,  1.5062],
        [ 0.5288,  0.3397],
        [ 0.9145, -1.0186],
        [-0.2796, -2.7372],
        [-0.7930,  0.7574],
        [ 0.3355,  0.5108],
        [ 0.5873, -0.1782],
        [-0.9113, -0.5134],
        [-0.2550,  1.5216],
        [ 1.2380, -1.3620],
        [-1.0021,  0.7534],
        [ 1.3978,  0.7960],
        [ 0.2329, -0.3418],
        [-1.2324, -1.7205],
        [-0.8113,  0.6322]])

In [7]:
emb = C[X]
emb.shape

torch.Size([32, 3, 2])

In [18]:
W1 = torch.randn((6, 100))
b1 = torch.randn(100)

In [32]:
#emb @ W1 + b1 # not possible since emb.shape = (32, 3, 2) and W1.shape = (6,100)
# need to convert emb to a multipliable shape
emb.view(32,6).shape

torch.Size([32, 6])

In [46]:
h = torch.tanh(emb.view(-1, 6) @ W1 + b1)

In [48]:
h.shape

torch.Size([32, 100])

In [50]:
W2 = torch.randn((100, 27))
b2 = torch.randn(27)

In [52]:
logits = h @ W2 + b2

In [54]:
logits.shape

torch.Size([32, 27])

In [56]:
counts = logits.exp()

In [60]:
prob = counts/counts.sum(1, keepdims=True)
prob.shape

torch.Size([32, 27])

In [64]:
prob[0].sum()

tensor(1.)

In [70]:
Y, Y.shape

(tensor([ 5, 13, 13,  1,  0, 15, 12,  9, 22,  9,  1,  0,  1, 22,  1,  0,  9, 19,
          1,  2,  5, 12, 12,  1,  0, 19, 15, 16,  8,  9,  1,  0]),
 torch.Size([32]))

In [90]:
prob[torch.arange(32), Y] #iterates over all 32 rows and plucks out the corresponding probability as given by Y
#we have to make all these values closer to 1, that's what we'll train the net on

tensor([5.0192e-09, 6.1503e-04, 3.9809e-03, 4.2947e-02, 5.1461e-01, 9.6388e-01,
        1.5995e-13, 5.6282e-14, 2.3788e-12, 9.2026e-12, 7.0800e-07, 9.9999e-01,
        1.6409e-05, 2.0863e-07, 2.3874e-03, 9.3512e-06, 5.1606e-10, 8.7055e-08,
        4.9668e-11, 6.5816e-05, 1.0073e-10, 1.0349e-09, 9.4893e-10, 6.1675e-03,
        5.3279e-08, 2.5612e-07, 8.9780e-01, 3.6018e-07, 1.3198e-08, 7.8853e-08,
        8.3471e-05, 1.0000e+00])

In [76]:
loss = -prob[torch.arange(32), Y].log().mean() #negative log likelihood loss
loss

tensor(13.6729)

In [9]:
#-------making it respectable just like andrej did------------>

Xtr.shape, Ytr.shape # dataset

(torch.Size([182625, 3]), torch.Size([182625]))

In [11]:
g = torch.Generator().manual_seed(2147483647) # for reproducibility
C = torch.randn((27, 10), generator=g)
W1 = torch.randn((30, 200), generator=g)
b1 = torch.randn(200, generator=g)
W2 = torch.randn((200, 27), generator=g)
b2 = torch.randn(27, generator=g)
parameters = [C, W1, b1, W2, b2]

In [13]:
sum(p.nelement() for p in parameters) # no. of parameters in total

11897

In [15]:
lre = torch.linspace(-3, 0, 1000)
lrs = 10**lre

In [17]:
for p in parameters:
    p.requires_grad = True

In [19]:
lri = []
lossi = []
stepi = []

In [21]:
for i in range(200000):
  
  # minibatch construct
  ix = torch.randint(0, Xtr.shape[0], (32,))
  
  # forward pass
  emb = C[Xtr[ix]] # (32, 3, 10)
  h = torch.tanh(emb.view(-1, 30) @ W1 + b1) # (32, 200)
  logits = h @ W2 + b2 # (32, 27)
  loss = F.cross_entropy(logits, Ytr[ix])
  #print(loss.item())
  
  # backward pass
  for p in parameters:
    p.grad = None
  loss.backward()
  
  # update
  #lr = lrs[i]
  lr = 0.1 if i < 100000 else 0.01
  for p in parameters:
    p.data += -lr * p.grad

  # track stats
  #lri.append(lre[i])
  stepi.append(i)
  lossi.append(loss.log10().item())

print(loss.item())

2.189784526824951


In [25]:
emb = C[Xtr] # (32, 3, 2)
h = torch.tanh(emb.view(-1, 30) @ W1 + b1) # (32, 100)
logits = h @ W2 + b2 # (32, 27)
loss = F.cross_entropy(logits, Ytr)
loss

tensor(2.1070, grad_fn=<NllLossBackward0>)

In [27]:
emb = C[Xdev] # (32, 3, 2)
h = torch.tanh(emb.view(-1, 30) @ W1 + b1) # (32, 100)
logits = h @ W2 + b2 # (32, 27)
loss = F.cross_entropy(logits, Ydev)
loss

tensor(2.1519, grad_fn=<NllLossBackward0>)

In [33]:
# sample from the model
g = torch.Generator().manual_seed(2147483647 + 10)

for _ in range(20):
    
    out = []
    context = [0] * block_size # initialize with all ...
    while True:
      emb = C[torch.tensor([context])] # (1,block_size,d)
      h = torch.tanh(emb.view(1, -1) @ W1 + b1)
      logits = h @ W2 + b2
      probs = F.softmax(logits, dim=1)
      ix = torch.multinomial(probs, num_samples=1).item()
      context = context[1:] + [ix]
      out.append(ix)
      if ix == 0:
        break
    
    print(''.join(itos[i] for i in out))

paya.
amorrosoleer.
math.
nili.
kiiyahny.
takaaron.
adon.
sivahs.
kaphiya.
safailiya.
danyam.
blee.
yulson.
makemijaeiren.
ostophesin.
jhavire.
majande.
josie.
euvin.
cyleas.
